# W266 Final Project - Feature_Sentence_Extraction

In [8]:
# Import a bunch of libraries.
import os
import sys
import glob
import time
from IPython.core.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import brown
nltk.download('brown')
from nltk.corpus import webtext
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import itertools

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mfrazzini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package brown to /home/mfrazzini/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/mfrazzini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mfrazzini/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#Dev:
#review_sentences = pd.read_csv('../data/Review_sentences_Dev_Active_Cleansed_2018-07-09.csv')
#Prod:
review_sentences = pd.read_csv('../data/Review_sentences_Prod_Active_Cleansed_2018-07-09.csv')

print(review_sentences.shape)
print(review_sentences.loc[0])
print(review_sentences.loc[1])
features = pd.read_csv('../data/FeatureDump_All_Active_2018-07-09T16_31_17.csv')
print(features.shape)
print(features.loc[0])
print(features.loc[1])


(1070247, 4)
Unnamed: 0                     0
SENTENCE_ID                    0
SENTENCE          Great suitcase
TESTIMONIAL_ID           1369284
Name: 0, dtype: object
Unnamed: 0                                                        1
SENTENCE_ID                                                       1
SENTENCE          I am no longer afraid of my stuff being squash...
TESTIMONIAL_ID                                              1369284
Name: 1, dtype: object
(212887, 3)
MODELID                              9863
FEATUREID                            3387
FEATURENAME    Removable checkbook holder
Name: 0, dtype: object
MODELID                    9863
FEATUREID                  3388
FEATURENAME    Bill compartment
Name: 1, dtype: object


In [3]:
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
 
def word_feats(words):
    return dict([(word, True) for word in words])

#Load subjective lines into a list
s = open(r'../data/quote.tok.gt9.5000', encoding = "ISO-8859-1")
subtxt = s.readlines()
#Append with actual bag feature list to dramatically improve accuracy
for index, row in features.iterrows():
    if type(row['FEATURENAME']) is str:
        subtxt.append(row['FEATURENAME'])
 

#Load neutral lines into a list
n = open(r'../data/plot.tok.gt9.5000', encoding = "ISO-8859-1")
neutxt = n.readlines()


subfeats = [(word_feats(nltk.word_tokenize(subtxt[x])), 'sub') for x in range(0,len(subtxt))]
neufeats = [(word_feats(nltk.word_tokenize(neutxt[x])), 'neu') for x in range(0,len(neutxt))]
 
subcutoff = int(len(subfeats)*3/4)
neucutoff = int(len(neufeats)*3/4)
 
trainfeats = subfeats[:subcutoff] + neufeats[:neucutoff]
testfeats = subfeats[subcutoff:] + neufeats[neucutoff:]
print('train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats)))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
classifier.show_most_informative_features()


train on 167164 instances, test on 55722 instances
accuracy: 0.9609310505724848
Most Informative Features
              girlfriend = True              neu : sub    =   1031.1 : 1.0
                 decides = True              neu : sub    =    845.2 : 1.0
               detective = True              neu : sub    =    827.7 : 1.0
                 married = True              neu : sub    =    769.7 : 1.0
                daughter = True              neu : sub    =    692.2 : 1.0
               boyfriend = True              neu : sub    =    653.5 : 1.0
                    kill = True              neu : sub    =    618.6 : 1.0
                    nick = True              neu : sub    =    566.4 : 1.0
                  leader = True              neu : sub    =    508.3 : 1.0
                   plans = True              neu : sub    =    479.2 : 1.0


In [4]:
import csv
with open('Feature_sentences_Prod_Active_2018-07-09.csv','w') as csvfile:
    fieldnames=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)

with open('NotFeature_sentences_Prod_Active_2018-07-09.csv','w') as csvfile2:
    fieldnames=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE']
    writer=csv.writer(csvfile2)
    writer.writerow(fieldnames)

sentences = 0
subjective = 0
neutral = 0
feature_sentences = pd.DataFrame()
not_feature_sentences = pd.DataFrame()
for index, row in review_sentences.iterrows():
    sentences += 1
    if type(row['SENTENCE']) is str:
        if (classifier.prob_classify(word_feats(nltk.word_tokenize(row['SENTENCE']))).prob('sub')) > .9:
            subjective += 1
            #print(row['SENTENCE'])
            feature_sentences = pd.DataFrame(columns=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE'])
            feature_sentences.loc[0,'TESTIMONIAL_ID'] = row['TESTIMONIAL_ID']
            feature_sentences.loc[0,'SENTENCE_ID'] = row['SENTENCE_ID']
            feature_sentences.loc[0,'SENTENCE'] = row['SENTENCE']
            feature_sentences.to_csv("Feature_sentences_Prod_Active_2018-07-09.csv",mode='a', header=False, index=False)
        else:
            neutral += 1
            not_feature_sentences = pd.DataFrame(columns=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE'])
            not_feature_sentences.loc[0,'TESTIMONIAL_ID'] = row['TESTIMONIAL_ID']
            not_feature_sentences.loc[0,'SENTENCE_ID'] = row['SENTENCE_ID']
            not_feature_sentences.loc[0,'SENTENCE'] = row['SENTENCE']
            not_feature_sentences.to_csv("NotFeature_sentences_Prod_Active_2018-07-09.csv",mode='a', header=False, index=False)
            
print("Sentences:", sentences, "Subjective:", subjective, "Neutral:", neutral)

Sentences: 1070247 Subjective: 333050 Neutral: 736136


In [5]:
feature_text = word_tokenize(features.loc[0]['FEATURENAME'])
tagged_text = nltk.pos_tag(feature_text)
print(tagged_text)
print(feature_sentences.shape)

[('Removable', 'JJ'), ('checkbook', 'NN'), ('holder', 'NN')]
(1, 3)


In [6]:
with open('Feature_sentences_Prod_Final_2018-07-09.csv','w') as csvfile:
    fieldnames=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE']
    writer=csv.writer(csvfile)
    writer.writerow(fieldnames)
    
with open('NotFeature_sentences_Prod_Final_2018-07-09.csv','w') as csvfile2:
    fieldnames=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE']
    writer=csv.writer(csvfile2)
    writer.writerow(fieldnames)

feature_sentences = pd.read_csv('../product_feature_sentences/Feature_sentences_Prod_Active_2018-07-09.csv')
sentences = 0
pattern_match = 0
no_pattern_match = 0

def pos_pattern_search(text, grammar):
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar)
    tagged_sents = nltk.pos_tag_sents(nltk.word_tokenize(sent) for sent in nltk.sent_tokenize(text))
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower()
                  #for key, group in itertools.groupby(all_chunks, lambda (word,pos,chunk): chunk != 'O') if key]
                  #Python3:
                  for key, group in itertools.groupby(all_chunks, lambda word__pos__chunk: word__pos__chunk[2] != 'O') if key]
    return [cand for cand in candidates]

for index, row in feature_sentences.iterrows():
    sentences += 1
    if type(row['SENTENCE']) is str:
        if (len(pos_pattern_search(row['SENTENCE'], 'KT: {(<.+> <.+> <.+>)}')) > 0):
            if (len(pos_pattern_search(row['SENTENCE'], 'KT: {(<JJ> <NN|NS> <.*>*)+}')) > 0
            or len(pos_pattern_search(row['SENTENCE'], 'KT: {(<RB|RBR|RBS> <JJ> <C.*|D.*|E.*|I.*|J.*|l.*|NNP|P.*|R.*|T.*|U.*|V.*|W.*>)}')) > 0
            or len(pos_pattern_search(row['SENTENCE'], 'KT: {(<JJ> <JJ> <C.*|D.*|E.*|I.*|J.*|l.*|NNP|P.*|R.*|T.*|U.*|V.*|W.*>)}')) > 0
            or len(pos_pattern_search(row['SENTENCE'], 'KT: {(<NN|NNS> <JJ> <C.*|D.*|E.*|I.*|J.*|l.*|NNP|P.*|R.*|T.*|U.*|V.*|W.*>)}')) > 0
            or len(pos_pattern_search(row['SENTENCE'], 'KT: {(<RB|RBR|RBS> <VB|VBN|VBD|VBG> <.*>)}')) > 0 ):
                pattern_match += 1
                #print(row['SENTENCE'])
                #print(nltk.pos_tag(word_tokenize(row['SENTENCE']))) 
                feature_sentences_final = pd.DataFrame(columns=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE'])
                feature_sentences_final.loc[0,'TESTIMONIAL_ID'] = row['TESTIMONIAL_ID']
                feature_sentences_final.loc[0,'SENTENCE_ID'] = row['SENTENCE_ID']
                feature_sentences_final.loc[0,'SENTENCE'] = row['SENTENCE']
                feature_sentences_final.to_csv("Feature_sentences_Prod_Final_2018-07-09.csv",mode='a', header=False, index=False)

            else:
                no_pattern_match +=1
                not_feature_sentences_final = pd.DataFrame(columns=['TESTIMONIAL_ID','SENTENCE_ID','SENTENCE'])
                not_feature_sentences_final.loc[0,'TESTIMONIAL_ID'] = row['TESTIMONIAL_ID']
                not_feature_sentences_final.loc[0,'SENTENCE_ID'] = row['SENTENCE_ID']
                not_feature_sentences_final.loc[0,'SENTENCE'] = row['SENTENCE']
                not_feature_sentences_final.to_csv("NotFeature_sentences_Prod_Final_2018-07-09.csv",mode='a', header=False, index=False)
        else:
            no_pattern_match +=1
            #print(row['SENTENCE'])
            #print(nltk.pos_tag(word_tokenize(row['SENTENCE']))) 
                
print("Sentences:", sentences, "Pattern Match:", pattern_match, "No Pattern Match:", no_pattern_match)

Sentences: 333050 Pattern Match: 146018 No Pattern Match: 187032


In [14]:
# Sentence extraction validation
path =r'..' # use your path
allFiles = glob.glob(path + "/data/RR_Pared_All_Active_SBD_2018-07-09_*.csv")
all_reviews = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
all_reviews = pd.concat(list_)

print(all_reviews.shape)

validation_sample = all_reviews.sample(n=1000)
print(validation_sample.shape)
validation_sample.to_csv("Validation_sentences_Prod_Active_2018-07-09.csv", index=False)

(1070247, 3)
(1000, 3)


In [63]:
# Load labeled data set
validated_sentences = pd.read_csv('Validation_sentences_Prod_Active_2018-07-09_Labeled_1K.csv', encoding='utf-8')
# Distribution by Sentiment Class
print(validated_sentences.groupby('INTERESTING').size())

# Load pattern match final dataset
pattern_match_final = pd.read_csv('Feature_sentences_Prod_Final_2018-07-09.csv', encoding='utf-8')
print(pattern_match_final.shape)

INTERESTING
0    804
1    196
dtype: int64
(146018, 3)


In [64]:
# Have to estimate false positives:
#  assuming ratio in pattern_match_final to total is same as ratio of validation set
positive_ratio_from_validation_set = (196/804) # 
print(positive_ratio_from_validation_set)
positive_ratio_from_proccessing = (pattern_match_final.shape[0]/1070247)
print(positive_ratio_from_proccessing)
est_false_positives = 218 * (positive_ratio_from_validation_set - positive_ratio_from_proccessing)
print(est_false_positives)
#print(pattern_match_final.shape[0])

0.24378109452736318
0.13643392600025975
23.401682738908548


In [65]:
true_positives = 0
false_negatives = 0
selected_items = 0


for index, row in validated_sentences.iterrows():
    if row['INTERESTING'] == 1:
        selected_items += 1
        if pattern_match_final[(pattern_match_final['TESTIMONIAL_ID'] == row['TESTIMONIAL_ID']) \
                              & (pattern_match_final['SENTENCE_ID'] == row['SENTENCE_ID'])].empty:
            false_negatives += 1
        else:
            true_positives += 1
            
            
print("Selected Items:", selected_items, "True Positives:", true_positives, "False Negatives:", false_negatives)
print("  Estimated Precision=", (true_positives/(true_positives+est_false_positives)))
print("  Recall=", (true_positives/(true_positives+false_negatives)))

Selected Items: 196 True Positives: 29 False Negatives: 167
  Estimated Precision= 0.5534173424256724
  Recall= 0.14795918367346939
